# 2 Using Command Line Tools

Objectives:
 * Learn about the command line tools that are included with the HDF5 library (and h5pyd package for HSDS)
 * You can run these in a notebook using the shell escape '!', or open a terminal window and run them from there
 

In [1]:
# get a sample HDF5 file
! wget https://s3.amazonaws.com/hdfgroup/data/hdf5test/tall.h5

[0] Downloading 'https://s3.amazonaws.com/hdfgroup/data/hdf5test/tall.h5' ...
Saving 'tall.h5'
HTTP response 200 OK [https://s3.amazonaws.com/hdfgroup/data/hdf5test/tall.h5]


In [2]:
# Display objects with a file. Use -r for recursive mode
! h5ls -r tall.h5

/                        Group
/g1                      Group
/g1/g1.1                 Group
/g1/g1.1/dset1.1.1       Dataset {10, 10}
/g1/g1.1/dset1.1.2       Dataset {20}
/g1/g1.2                 Group
/g1/g1.2/extlink         External Link {somefile//somepath}
/g1/g1.2/g1.2.1          Group
/g1/g1.2/g1.2.1/slink    Soft Link {somevalue}
/g2                      Group
/g2/dset2.1              Dataset {10}
/g2/dset2.2              Dataset {3, 5}


In [3]:
# h5dump will show the dataset contents by default
! h5dump -r tall.h5

HDF5 "tall.h5" {
GROUP "/" {
   ATTRIBUTE "attr1" {
      DATATYPE  H5T_STD_I8BE
      DATASPACE  SIMPLE { ( 10 ) / ( 10 ) }
      DATA {
         "abcdefghi\000"
      }
   }
   ATTRIBUTE "attr2" {
      DATATYPE  H5T_STD_I32BE
      DATASPACE  SIMPLE { ( 2, 2 ) / ( 2, 2 ) }
      DATA {
      (0,0): 0, 1,
      (1,0): 2, 3
      }
   }
   GROUP "g1" {
      GROUP "g1.1" {
         DATASET "dset1.1.1" {
            DATATYPE  H5T_STD_I32BE
            DATASPACE  SIMPLE { ( 10, 10 ) / ( 10, 10 ) }
            DATA {
            (0,0): 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            (1,0): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
            (2,0): 0, 2, 4, 6, 8, 10, 12, 14, 16, 18,
            (3,0): 0, 3, 6, 9, 12, 15, 18, 21, 24, 27,
            (4,0): 0, 4, 8, 12, 16, 20, 24, 28, 32, 36,
            (5,0): 0, 5, 10, 15, 20, 25, 30, 35, 40, 45,
            (6,0): 0, 6, 12, 18, 24, 30, 36, 42, 48, 54,
            (7,0): 0, 7, 14, 21, 28, 35, 42, 49, 56, 63,
            (8,0): 0, 8, 16, 24, 32, 40, 48, 

In [4]:
# On the HSDS side, hsinfo will show the current server status
! hsinfo

server name: hsds on devone
server state: READY
endpoint: http://hsds.hdf.test:5101
username: test_user1 
password: ****
home: /home/test_user1/
server version: 0.8.1
node count: 1
up: 59 min 48 sec
h5pyd version: 0.15.1


In [5]:
# you can use hsload to copy a local file to HSDS
! hsload tall.h5 hdf5://home/test_user1/

In [6]:
# hsls works like h5ls but with content managed by the server
! hsls -r hdf5://home/test_user1/tall.h5

/ Group
/g1 Group
/g1/g1.1 Group
/g1/g1.1/dset1.1.1 Dataset {10, 10}
/g1/g1.1/dset1.1.2 Dataset {20}
/g1/g1.2 Group
/g1/g1.2/g1.2.1 Group
/g1/g1.2/g1.2.1/slink    SoftLink {somevalue}
/g1/g1.2/extlink         ExternalLink {somepath//somefile}
/g2 Group
/g2/dset2.1 Dataset {10}
/g2/dset2.2 Dataset {3, 5}


In [7]:
# hsls can also be used to display contents of an HSDS folder
# Note: trailing slash is required
! hsls hdf5://home/test_user1/

test_user1                                          folder   2023-03-10 17:20:29 hdf5://home/test_user1/
test_user1                                          domain   2023-08-09 16:59:50 hdf5://home/test_user1/01.h5
test_user1                                          domain   2023-08-09 16:39:16 hdf5://home/test_user1/03.h5
test_user1                                          domain   2023-08-09 18:39:25 hdf5://home/test_user1/04.h5
test_user1                                          domain   2023-08-10 17:21:23 hdf5://home/test_user1/05.h5
test_user1                                          domain   2023-08-10 17:37:10 hdf5://home/test_user1/06.h5
test_user1                                          domain   2023-04-11 16:47:20 hdf5://home/test_user1/ascii_attr.h5
test_user1                                          domain   2023-04-10 12:43:53 hdf5://home/test_user1/d_nullref.h5
test_user1                                          domain   2023-03-22 21:37:59 hdf5://home/test_user1/d_objr

In [8]:
# hsstat provides information about storage used for HSDS objects similar to how h5stat works
# Note: it may take a minute before the information displayed by hsstat is current
! hsstat hdf5://home/test_user1/tall.h5

domain: hdf5://home/test_user1/tall.h5
    owner:           test_user1
    id:              g-d761f590-972a9c10-5fe9-7d181f-a21498
    last modified:   2023-08-10 17:54:20
    total_size:      0
    allocated_bytes: 0
    num objects:     0
    num chunks:      0


Problem: If the path name doesn't end in a slash, hsls assumes you are looking for a domain, not a folder.  What does "hsls -H -v /shared" return?

Problem: Try the above command with the -H -v flags

Problem: Try the above command with different options:
* -v
* --showattrs
* --showacls
* --loglevel debug

Problem: List the contents of the home directory for your account.
Add the --showacls flag to show the permissions for this folder

Problem: Run ``$showacl /home/test_user1/`` to show the permissions of the folder

### Upload the file to your home folder
Run: `$ hsload tall.h5 /home/test_user1/`  # replace 'myfolder' with your actual folder


### List the contents of the uploaded file
The file is now stored in HSDS, use hsls to display it:
`$ hsls -r /home/test_user1/tall.h5`



### Download the file 
Run: `$ hsget /home/test_user1/tall.h5 tall2.h5`
This will download the file as tall2.h5


### Compare the two files
Run: `$ h5dump tall.h5` 
and`$h5dump tall2.h5`

Problem: Are these files the same?

### ACLS (Access Control List)
Each server domain or folder can contain one or more ACLs that control
who may perform operations (e.g. read/update/delete)

Run: $ hsacl /home/test_user1/tall.h5

Problem: What happens when you run $ hsacl /home/test_user1/tall.h5?

Problem: Update *your* tall.h5 so that anyone can read the ACLs
(see hsacl --help)